In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import pickle
import sys

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pprint

project_root = '..'
sys.path.append(project_root)

from sleep.data.loader import load_dataset
from sleep.detection.feeder_dataset import FeederDataset
from sleep.detection.predicted_dataset import PredictedDataset
from sleep.detection import metrics
from sleep.common import constants, pkeys, checks
from sleep.data import utils

%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

RESULTS_PATH = os.path.join(project_root, 'results')
SEED_LIST = [123, 234, 345, 456]

In [2]:
ckpt_folder = '20190504_bsf'
task_mode = constants.WN_RECORD
dataset_name = constants.MASS_SS_NAME
seed_id = 0
chosen_thr = 0.5
which_expert = 1
verbose = False

dataset = load_dataset(dataset_name)
all_train_ids = dataset.train_ids
full_ckpt_folder = '%s_%s_train_%s' % (ckpt_folder, task_mode, dataset_name)
ckpt_path = os.path.abspath(os.path.join(
    RESULTS_PATH,
    'predictions_%s' % dataset_name,
    full_ckpt_folder,
    'bsf',
    'seed%d' % seed_id
))
filename = os.path.join(
    ckpt_path,
    'prediction_%s_%s.pkl' % (task_mode, constants.VAL_SUBSET))
with open(filename, 'rb') as handle:
    prediction_val = pickle.load(handle)  # Predicted Dataset
    
# Validation split
_, val_ids = utils.split_ids_list(
    all_train_ids, seed=SEED_LIST[seed_id], verbose=verbose)
# Prepare expert labels
data_val = FeederDataset(
    dataset, val_ids, task_mode, which_expert=which_expert)
# Prepare model predictions
prediction_val.set_probability_threshold(chosen_thr)

print('Val IDs:', data_val.get_ids(), prediction_val.get_ids())

Train size: 11. Test size: 4
Train subjects: 
 [1, 3, 5, 7, 9, 10, 11, 14, 17, 18, 19]
Test subjects: 
 [2, 6, 12, 13]
Dataset mass_ss with 15 patients.
Loading from checkpoint... Loaded
Val IDs: [5, 11, 19] [5, 11, 19]


In [ ]:
# Get stamps
events_val = data_val.get_stamps()
detections_val = prediction_val.get_stamps()